In [1]:
import numpy as np

class U_net():
    def __init__(self, n_class):
        super().__init__()
        self._weights = []
        self._bias = []
        self._matrixes = []
        self.n_class = n_class
        
    def preprocessing(self, img):
        result = np.zeros_like(img, dtype=float)
        for channel in range(img.shape[-1]):
            min_ = np.min(img[...,channel])
            max_ = np.max(img[...,channel])
            result[...,channel] = (img[...,channel] - min_)/(max_ - min_)
        return result
    
    def convalutionForward(self, img, num_filt):
        # convalution with RELU
        n_filt = img.shape[2]
        filters_3x3xn = [np.random.normal(loc=0.5, scale=0.15, size=(3,3,n_filt)) for i in range(num_filt)]
        filters_3x3xn = np.array(filters_3x3xn)
        bias = np.zeros((num_filt, 1))
        # в некоторых вариантах U-net не используются падинги
        out = np.zeros((img.shape[0]-2, img.shape[0]-2, num_filt))
        for channel in range(num_filt):
            for row in range(1, img.shape[0] - 1):
                for col in range(1, img.shape[1] - 1):
                    result = np.sum(img[row-1: row+2, col-1: col+2, :]*filters_3x3xn[channel]+bias[channel])
                    out[row-1, col-1, channel] = np.maximum(result, 0) # RELU
        self._weights.append(filters_3x3xn)
        self._bias.append(bias)
        return out
    
    def maxpool(self, image, f=2, s=2):
        h_prev, w_prev, n_c = image.shape
        h = int((h_prev - f)/s)+1
        w = int((w_prev - f)/s)+1
        downsampled = np.zeros((h, w, n_c))
        for i in range(n_c):
            curr_y = out_y = 0
            while curr_y + f <= h_prev:
                curr_x = out_x = 0
                while curr_x + f <= w_prev:
                    downsampled[out_y, out_x, i] = np.max(image[curr_y:curr_y+f, curr_x:curr_x+f, i])
                    curr_x += s
                    out_x += 1
                curr_y += s
                out_y += 1
        return downsampled
    
    def trans_concat(self, matrix_1, matrix_2):
        diff = matrix_1.shape[0] - matrix_2.shape[0]
        diff_s = int(diff/2)
        diff_f = int(matrix_1.shape[0]-diff_s)
        if diff % 2 == 1:
            diff_f -= 1
        matr_1_dif = matrix_1[diff_s:diff_f, diff_s:diff_f, :]
        result = np.concatenate((matr_1_dif, matrix_2), axis=2)
        return result
    
    def unpooling(self, matrix):
        matrix_m, matrix_n, channel = matrix.shape
        kernel = np.ones((2, 2), dtype=np.float)
        kernel_m, kernel_n = kernel.shape
        unpooled_matrix = np.zeros((matrix_m * kernel_m, matrix_n * kernel_n, channel), dtype=np.float)
        for ch in range(channel):
            for i in range(matrix_m):
                pos_i = i * kernel_m
                for j in range(matrix_n):
                    pos_j = j * kernel_n
                    unpooled_matrix[pos_i: pos_i + kernel_m, pos_j: pos_j + kernel_n, ch] = kernel * matrix[i][j][ch]
        return (unpooled_matrix)
    
    def finish_layer(self, img, num_class):
        n_filt = img.shape[2]
        filters_1x1xn = [np.random.normal(loc=0.5, scale=0.15, size=(1,1,n_filt)) for i in range(num_class)]
        filters_1x1xn = np.array(filters_1x1xn)
        bias = np.zeros((num_class, 1))
        result = np.zeros((img.shape[0], img.shape[0], num_class))
        out = np.zeros((img.shape[0], img.shape[0], num_class))
        for channel in range(num_class):
            for row in range(img.shape[0]):
                for col in range(img.shape[1]):
                    result[row, col, channel] = np.sum(img[row, col, :]*filters_1x1xn[channel]+bias[channel])
        for channel in range(num_class):
            for row in range(img.shape[0]):
                for col in range(img.shape[1]):
                    # SoftMax
                    out[row, col, channel] = np.exp(result[row, col, channel])/np.sum(np.exp(result[row, col, :]))
        self._weights.append(filters_1x1xn)
        self._bias.append(bias)
        return out
        
    def forward(self, img):
        step_1 = self.preprocessing(img)
        step_2 = self.convalutionForward(step_1, 64)
        step_3 = self.preprocessing(step_2)
        step_4 = self.convalutionForward(step_3, 64)
        step_5 = self.preprocessing(step_4)
        step_6 = self.maxpool(step_5)
        step_7 = self.convalutionForward(step_6, 128)
        step_8 = self.preprocessing(step_7)
        step_9 = self.convalutionForward(step_8, 128)
        step_10 = self.preprocessing(step_9)
        step_11 = self.maxpool(step_10)
        step_12 = self.convalutionForward(step_11, 256)
        step_13 = self.preprocessing(step_12)
        step_14 = self.convalutionForward(step_13, 256)
        step_15 = self.preprocessing(step_14)
        step_16 = self.maxpool(step_15)
        step_17 = self.convalutionForward(step_16, 512)
        step_18 = self.preprocessing(step_17)
        step_19 = self.convalutionForward(step_18, 512)
        step_20 = self.preprocessing(step_19)
        step_21 = self.maxpool(step_20)
        step_22 = self.convalutionForward(step_21, 1024)
        step_23 = self.preprocessing(step_22)
        step_24 = self.convalutionForward(step_23, 1024)
        step_25 = self.preprocessing(step_24)
        step_26 = self.unpooling(step_25)
        step_27 = self.trans_concat(step_20, step_26)
        step_28 = self.convalutionForward(step_27, 512)
        step_29 = self.preprocessing(step_28)
        step_30 = self.convalutionForward(step_29, 512)
        step_31 = self.preprocessing(step_30)
        step_32 = self.unpooling(step_31)
        step_33 = self.trans_concat(step_15, step_32)
        step_34 = self.convalutionForward(step_33, 256)
        step_35 = self.preprocessing(step_34)
        step_36 = self.convalutionForward(step_35, 256)
        step_37 = self.preprocessing(step_36)
        step_38 = self.unpooling(step_37)
        step_39 = self.trans_concat(step_10, step_38)
        step_40 = self.convalutionForward(step_39, 128)
        step_41 = self.preprocessing(step_40)
        step_42 = self.convalutionForward(step_41, 128)
        step_43 = self.preprocessing(step_42)
        step_44 = self.unpooling(step_43)
        step_45 = self.trans_concat(step_5, step_44)
        step_46 = self.convalutionForward(step_45, 64)
        step_47 = self.preprocessing(step_46)
        step_48 = self.convalutionForward(step_47, 64)
        step_49 = self.preprocessing(step_48)
        step_50 = self.finish_layer(step_49, self.n_class)
        return step_50

    def jaccard_coef(self, y_true, y_pred):
        y_true = np.reshape(y_true, (-1, 1))
        y_pred = np.reshape(y_pred, (-1, 1))
        eps = 1e-7
        intersection = np.sum(y_true * y_pred)
        union = np.sum(y_true) + np.sum(y_pred) + eps
        return intersection / (union - intersection)


    def jaccard_loss(self, y_true, y_pred):
        return 1. - self.jaccard_coef(y_true, y_pred)

    def convolutionBackward(self, dconv_prev, conv_in, filt):
        (n_f, n_c, f, n_f) = filt.shape
        (_, orig_dim, _) = conv_in.shape
        dout = np.zeros(conv_in.shape) 
        dfilt = np.zeros(filt.shape)
        dbias = np.zeros((n_f,1))
        for channel in range(num_filt):
            for row in range(1, img.shape[0] - 1):
                for col in range(1, img.shape[1] - 1):
                    dfilt[..., channel] += dconv_prev[row, col, channel] * conv_in[row-1: row+2, col-1: col+2, :]
                    dout[row-1: row+2, col-1: col+2, :] += dconv_prev[row, col, channel] * filt[channel] 
            dbias[curr_f] = np.sum(dconv_prev[curr_f])
        return dout, dfilt, dbias
    
    def maxpoolBackward(self, dpool, orig, f=2, s=2):
        (n_c, orig_dim, _) = orig.shape
        dout = np.zeros(orig.shape)
        for curr_c in range(n_c):
            curr_y = out_y = 0
            while curr_y + f <= orig_dim:
                curr_x = out_x = 0
                while curr_x + f <= orig_dim:
                    (a, b) = nanargmax(orig[curr_c, curr_y:curr_y+f, curr_x:curr_x+f])
                    dout[curr_c, curr_y+a, curr_x+b] = dpool[curr_c, out_y, out_x]

                    curr_x += s
                    out_x += 1
                curr_y += s
                out_y += 1
        return dout

In [2]:
import cv2
img = cv2.imread('lena.png')

In [3]:
ddf = U_net(n_class=2)

In [4]:
result = ddf.forward(img)

In [5]:
img.shape, result.shape

((512, 512, 3), (324, 324, 2))